In [ ]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps


#### FIXED #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
credentials={'username' : "admin",
                'password' : "password"}
shelter = AnimalShelter(credentials)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))
# Logo 
image_filename = 'Grazioso_Salvare_Logo.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
#########################
# Dashboard Layout / View
#########################
app = dash.Dash()


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A( html.Center(html.Img(style={'width':'300px', 'height': '300px'},alt="Company Logo", src='data:image/png;base64,{}'.format(encoded_image.decode()))),
           href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('Animal Shelter Outcomes Dashboard'))),
    html.Center(html.B(html.H2('Project By: Gino Murin'))),
    html.Center(html.B(html.H3('SNHU CS-340'))),
    html.Hr(),
    html.Div(
        #Radio Items to select the rescue filter options
        dcc.RadioItems(
            id='filter-type',
            # created the labels and keys based on the Grazioso requirements
            options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain Rescue', 'value': 'MWR'},
            {'label': 'Disaster Rescue', 'value': 'DRIT'},
            {'label': 'Reset', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block','font-family':'Open Sans, sans-serif',
                'font-size': '16px'}
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        #made selectable 'multi' to allow map to work with several options
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
                    style_header={
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white',
        'font-family':'Open Sans, sans-serif',
                'font-size': '16px'
    },
    style_data={
        'backgroundColor': 'rgb(50, 50, 50)',
        'color': 'white',
        'font-family':'Open Sans, sans-serif',
        'font-size': '16px'
    },
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            #dcc.Graph(id='graph-id'),
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        if filter_type == 'WR':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Labrador Retriever Mix'},
                                                              {'breed': 'Chesa Bay Retr Mix'},
                                                              {'breed': 'Newfoundland'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))
        elif filter_type == 'MWR':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'Rottweiler'},
                                                              {'breed': 'Siberian Husky'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))
        elif filter_type == 'DRIT':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinscher'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Bloodhound'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]
                                                })))
        elif filter_type == 'RESET':
            df = pd.DataFrame.from_records(shelter.read({}))

        
        columns=[{"name":  i.title().replace("_", " "), "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#function to update the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names (breed) and values (recurring counts)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])
def update_map(viewData, selected_rows, selected_columns):
    # imports the currently viewed data
    dff = pd.DataFrame.from_dict(viewData)
    #determines the default status
    if selected_rows == []:
        selected_rows = [0]
    # creats a map for a single selected row or the default
    if len(selected_rows) == 1:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ])
            ])
        ]
    #creats a map with 2 markers
    elif len(selected_rows) == 2:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ])
            ])
        ]
    #creats a map with 3 markers
    elif len(selected_rows) == 3:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[2],13]), (dff.iloc[selected_rows[2],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[2],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[2],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[2],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[2],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[2],15])
                    ])
                ])
            ])
        ]
    #creats a map with 4 markers
    elif len(selected_rows) == 4:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[2],13]), (dff.iloc[selected_rows[2],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[2],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[2],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[2],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[2],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[2],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[3],13]), (dff.iloc[selected_rows[3],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[3],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[3],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[3],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[3],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[3],15])
                    ])
                ])
            ])
        ]
    #creats a map with 5 markers
    elif len(selected_rows) == 5:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[2],13]), (dff.iloc[selected_rows[2],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[2],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[2],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[2],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[2],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[2],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[3],13]), (dff.iloc[selected_rows[3],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[3],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[3],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[3],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[3],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[3],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[4],13]), (dff.iloc[selected_rows[4],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[4],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[4],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[4],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[4],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[4],15])
                    ])
                ])
            ])
        ]
    
app.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Apr/2022 18:53:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:53:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:53:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:53:16] "GET /env HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:53:16] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:53:16] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:53:16] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dash/dash.py", line 1344, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "

127.0.0.1 - - [17/Apr/2022 18:53:16] "POST /_dash-update-component HTTP/1.1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3621, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'breed'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Library/Frameworks/Python.framework/Vers

...

127.0.0.1 - - [17/Apr/2022 18:54:12] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Apr/2022 18:54:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:54:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:54:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:54:12] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 18:54:13] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:07] "

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dash/dash.py", line 1344, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "

127.0.0.1 - - [17/Apr/2022 19:01:22] "POST /_dash-update-component HTTP/1.1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3621, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'breed'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Library/Frameworks/Python.framework/Vers

" 500 -
127.0.0.1 - - [17/Apr/2022 19:01:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Apr/2022 19:01:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:23] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2022 19:01:23] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
